In [1]:
from utils import *

## Identify dup in t_zi_part table 

In [4]:
sql_stmt = """
with dup as (
	select u_id,count(*) from t_zi_part 
	where u_id is not NULL 
	group by u_id having count(*) > 1
	order by cast(u_id as int)
) 
select p.* from t_zi_part p join dup on dup.u_id = p.u_id 
order by p.u_id;
"""

with DBConn() as _conn:
    df = pd.read_sql(sql_stmt, _conn)

In [5]:
df

,zi,u_id,zi_left_up,zi_left,zi_left_down,zi_up,zi_mid,zi_down,zi_right_up,zi_right,zi_right_down,zi_mid_out,zi_mid_in,ts,desc_cn,is_active,id_shuowen
0,朓,10081,None,肉,None,None,None,None,None,兆,None,None,None,None,祭也。从肉兆聲。,Y,2658
1,朓,10081,None,月,None,None,None,None,None,兆,None,None,None,None,晦而月見西方謂之朓。从月兆聲。,Y,4296
2,鳽,11560,None,隹,None,None,None,None,None,幵,None,None,None,None,石鳥。一名雝𪆫。一曰精𠛱。从隹幵聲。《春秋傳》：“秦有士鳽 。”,Y,2290
3,鳽,11560,None,鳥,None,None,None,None,None,幵,None,None,None,None,𪁉𪂴也。从鳥幵聲。,Y,2439
4,雁,11628,None,隹,None,None,None,None,None,人,None,None,None,None,鳥也。从隹从人，厂聲。讀若鴈。,Y,2293
5,雁,11628,None,人,None,None,None,None,None,厂,None,None,None,None,鳥也。从隹从人，厂聲。讀若鴈。,Y,2293
6,薻,12596,None,艸,None,None,None,None,None,水,None,None,None,None,水艸也。从艸从水，巢聲。《詩》曰：“于以采薻？”,Y,650
7,薻,12596,None,水,None,None,None,None,None,巢,None,None,None,None,水艸也。从艸从水，巢聲。《詩》曰：“于以采薻？”,Y,650
8,胄,13147,None,肉,None,None,None,None,None,由,None,None,None,None,𦙍也。从肉由聲。,Y,2636
9,胄,13147,None,冃,None,None,None,None,None,由,None,None,None,None,兜鍪也。从冃由聲。,Y,4785


In [6]:
df.to_csv("t_zi_part_dup.csv", index=False)

In [15]:
sql_stmt = """
select *
from t_zi_part --t_part -- t_zi
;
"""

with DBConn() as _conn:
    df = db_run_sql(sql_stmt, _conn)

[DEBUG] 
select *
from t_zi_part --t_part -- t_zi
;



In [16]:
df.columns 

Index(['zi', 'u_id', 'zi_left_up', 'zi_left', 'zi_left_down', 'zi_up',
       'zi_mid', 'zi_down', 'zi_right_up', 'zi_right', 'zi_right_down',
       'zi_mid_out', 'zi_mid_in', 'ts', 'desc_cn', 'is_active'],
      dtype='object')

In [17]:
print("\n, ".join(df.columns))

zi
, u_id
, zi_left_up
, zi_left
, zi_left_down
, zi_up
, zi_mid
, zi_down
, zi_right_up
, zi_right
, zi_right_down
, zi_mid_out
, zi_mid_in
, ts
, desc_cn
, is_active


In [18]:
sql_stmt = """
select *
from w_zi_part -- w_part -- w_zi 
limit 5;
"""

with DBConn() as _conn:
    df2 = db_run_sql(sql_stmt, _conn)

[DEBUG] 
select *
from w_zi_part -- w_part -- w_zi 
limit 5;



In [20]:
print("\n, ".join(df2.columns))

zi
, zi_left_up
, zi_left
, zi_left_down
, zi_up
, zi_mid
, zi_down
, zi_right_up
, zi_right
, zi_right_down
, zi_mid_out
, zi_mid_in
, id
, ts
, uid
, is_active
, shuowen_desc


In [19]:
df2

,zi,zi_left_up,zi_left,zi_left_down,zi_up,zi_mid,zi_down,zi_right_up,zi_right,zi_right_down,zi_mid_out,zi_mid_in,id,ts,uid,is_active,shuowen_desc
0,元,None,,None,一,None,兀,None,,None,None,None,2,None,None,1,始也。从一从兀。
1,吏,None,,None,一,史,None,None,,None,None,None,5,None,None,1,治人者也。从一从史，史亦聲。
2,禮,None,示,None,None,None,None,None,豊,None,None,None,12,None,None,0,履也。所以事神致福也。从示从豊，豊亦聲。
3,祰,None,示,None,None,None,None,None,告,None,None,None,38,None,None,0,告祭也。从示从告聲。
4,祏,None,示,None,None,None,None,None,石,None,None,None,39,None,None,1,宗廟主也。《周禮》有郊、宗、石室。一曰大夫以石爲主。从示从石，石亦聲。


In [2]:
sql_stmt = """
select zi,id,sort_val,layer 
from t_zi where as_part=1 order by sort_val;
"""

with DBConn() as _conn:
    df = db_run_sql(sql_stmt, _conn)

In [4]:
df

,zi,id,sort_val,layer
0,八,314,1.0,HSK_1-Common-01
1,贝,1574,2.0,HSK_1-Common-01
2,比,1633,3.0,HSK_1-Common-01
3,卜,1911,5.0,HSK_1-Common-01
4,长,2160,12.0,HSK_1-Common-01
...,...,...,...,...
202,爫,12796,11070.0,HSK_8
203,夂,12998,11191.0,HSK_8
204,豸,13063,11228.0,HSK_8
205,丶,13185,11308.0,HSK_8


In [5]:
df.to_csv("parts-v0.csv", index=False)

In [9]:
df = pd.read_csv("t_part.csv").fillna("")

In [11]:
df.columns

Index(['Radical number', 'Radical', 'Pinyin', 'Meaning', 'Strokes',
       'Traditional', 'Examples'],
      dtype='object')

In [13]:
col_map = {
    'Radical number': "id_radical", 
    'Radical':"radical", 
    'Pinyin':"pinyin", 
    'Meaning':"meaning", 
    'Strokes':"strokes",
    'Traditional':"traditional", 
    'Examples':"example"
}

In [14]:
df.rename(columns=col_map, inplace=True)

In [28]:
df

,id_radical,radical,pinyin,meaning,strokes,traditional,example,id,is_active
0,1,一,yī,one,1,,丁 万 上 下,1,1
1,2,丨,gǔn,vertical line,1,,个 中 丰 串 临,2,1
2,3,丶,zhǔ,dot,1,,丸 义 丹 为 主,3,1
3,4,丿,piě,slash,1,,乂 乃 乎,4,1
4,4,乀,piě,slash,1,,飞,5,1
...,...,...,...,...,...,...,...,...,...
253,210,齐,qí,neat,6,,挤 剂 济,254,1
254,211,齿,chǐ,tooth,8,,龄 龅,255,1
255,212,龙,lóng,dragon,5,,龚 笼 珑,256,1
256,213,龟,guī,turtle,7,,,257,1


In [18]:
df.columns

Index(['id_radical', 'radical', 'pinyin', 'meaning', 'strokes', 'traditional',
       'example', 'id'],
      dtype='object')

In [12]:
df.index

RangeIndex(start=0, stop=258, step=1)

In [16]:
df["id"] = df.index + 1

In [21]:
df["id_radical"] = df["id_radical"].map(lambda x: int(x) if x else -1)

In [22]:
df["strokes"] = df["strokes"].map(lambda x: int(x) if x else -1)

In [24]:
df["radical"] = df["radical"].str.strip()

In [25]:
df["meaning"] = df["meaning"].str.lower()

In [27]:
df["is_active"] = 1

In [35]:
t_str = """
        id int  NOT NULL,
        id_radical int,
        radical text NOT NULL,
        traditional text,
        pinyin text,
        strokes int,
        meaning text,
        example text,
        is_active int default 0
"""

In [36]:
cols = [i.strip().split()[0] for i in t_str.split("\n") if i.strip()]

In [37]:
cols

['id',
 'id_radical',
 'radical',
 'traditional',
 'pinyin',
 'strokes',
 'meaning',
 'example',
 'is_active']

In [38]:
df2 = df[cols].copy()

In [39]:
df2

,id,id_radical,radical,traditional,pinyin,strokes,meaning,example,is_active
0,1,1,一,,yī,1,one,丁 万 上 下,1
1,2,2,丨,,gǔn,1,vertical line,个 中 丰 串 临,1
2,3,3,丶,,zhǔ,1,dot,丸 义 丹 为 主,1
3,4,4,丿,,piě,1,slash,乂 乃 乎,1
4,5,4,乀,,piě,1,slash,飞,1
...,...,...,...,...,...,...,...,...,...
253,254,210,齐,,qí,6,neat,挤 剂 济,1
254,255,211,齿,,chǐ,8,tooth,龄 龅,1
255,256,212,龙,,lóng,5,dragon,龚 笼 珑,1
256,257,213,龟,,guī,7,turtle,,1


In [41]:
with DBConn() as _conn:
    df2.to_sql("t_part", _conn, if_exists='replace', index=False)

## table=t_zi

In [65]:
sql_stmt = """
select * from t_zi where as_part=1
"""

with DBConn() as _conn:
    df4 = pd.read_sql(sql_stmt, _conn)

df4.columns 

In [67]:
df4.head()

,zi,desc_cn,pinyin,nstrokes,alias,is_radical,is_zi,has_traditional,zi_en,desc_en,id,ts,uid,is_active,layer,is_picto,is_composed,as_part,sort_id,sort_val
0,石,None,dan4,None,None,1,1,NaN,None,dry measure for grain equal to ten dou 斗[dou3]...,183,None,None,1,HSK_1-Common-02,1.0,NaN,1,215,215.0
1,木,None,mu4,None,None,1,1,NaN,None,(bound form) tree/(bound form) wood/unresponsi...,189,None,None,1,HSK_1-Common-01,NaN,NaN,1,89,89.0
2,非,None,fei1,None,None,1,1,NaN,None,to not be; not/wrong,190,None,None,1,HSK_1-Common-06,NaN,NaN,1,939,939.0
3,隶,None,li4,None,隸,1,1,1.0,None,(bound form) a person in servitude; low-ranki...,216,None,None,1,HSK_1-Common-07,NaN,1.0,1,1172,1172.0
4,大,None,da4,None,None,1,1,NaN,None,big; large,224,None,None,1,HSK_1-Common-01,NaN,NaN,1,19,19.0


In [68]:
print("\n, ".join(df4.columns ))

zi
, desc_cn
, pinyin
, nstrokes
, alias
, is_radical
, is_zi
, has_traditional
, zi_en
, desc_en
, id
, ts
, uid
, is_active
, layer
, is_picto
, is_composed
, as_part
, sort_id
, sort_val


## table=t_zi_part

In [69]:
sql_stmt = """
select * from t_zi_part where is_active=1
"""

with DBConn() as _conn:
    df5 = pd.read_sql(sql_stmt, _conn)

df5.columns 

Index(['zi', 'zi_left_up', 'zi_left', 'zi_left_down', 'zi_up', 'zi_mid',
       'zi_down', 'zi_right_up', 'zi_right', 'zi_right_down', 'zi_mid_out',
       'zi_mid_in', 'id', 'ts', 'uid', 'is_active', 'shuowen_desc'],
      dtype='object')

In [70]:
print("\n, ".join(df5.columns ))

zi
, zi_left_up
, zi_left
, zi_left_down
, zi_up
, zi_mid
, zi_down
, zi_right_up
, zi_right
, zi_right_down
, zi_mid_out
, zi_mid_in
, id
, ts
, uid
, is_active
, shuowen_desc


In [75]:
print("\n OR ".join([f"{c} like '%{{search_term}}%'" for c in df5.columns if c.startswith("zi")] ))

zi like '%{search_term}%'
 OR zi_left_up like '%{search_term}%'
 OR zi_left like '%{search_term}%'
 OR zi_left_down like '%{search_term}%'
 OR zi_up like '%{search_term}%'
 OR zi_mid like '%{search_term}%'
 OR zi_down like '%{search_term}%'
 OR zi_right_up like '%{search_term}%'
 OR zi_right like '%{search_term}%'
 OR zi_right_down like '%{search_term}%'
 OR zi_mid_out like '%{search_term}%'
 OR zi_mid_in like '%{search_term}%'


In [71]:
df5.head()

,zi,zi_left_up,zi_left,zi_left_down,zi_up,zi_mid,zi_down,zi_right_up,zi_right,zi_right_down,zi_mid_out,zi_mid_in,id,ts,uid,is_active,shuowen_desc
0,元,None,,None,一,None,兀,None,,None,None,None,2,None,None,1,始也。从一从兀。
1,吏,None,,None,一,史,None,None,,None,None,None,5,None,None,1,治人者也。从一从史，史亦聲。
2,祏,None,示,None,None,None,None,None,石,None,None,None,39,None,None,1,宗廟主也。《周禮》有郊、宗、石室。一曰大夫以石爲主。从示从石，石亦聲。
3,祝,None,示,None,None,None,None,None,人,None,None,None,47,None,None,1,祭主贊詞者。从示从人口。一曰从兌省。《易》曰：“兌爲口爲巫。”。
4,祟,None,示,None,None,None,None,None,出,None,None,None,68,None,None,1,神禍也。从示从出。


## compare radicals

In [58]:
sql_stmt = """
--select zi from t_zi where as_part=1
--except
select radical as zi,traditional as zi_tr from t_part 
order by id_radical
"""

with DBConn() as _conn:
    df3 = pd.read_sql(sql_stmt, _conn)

In [60]:
df3["zi2"] = df3["zi"] + df3["zi_tr"]

In [61]:
df3

,zi,zi_tr,zi2
0,一,,一
1,丨,,丨
2,丶,,丶
3,丿,,丿
4,乀,,乀
...,...,...,...
251,齐,,齐
252,齿,,齿
253,龙,,龙
254,龟,,龟


In [76]:
parts  = df3["zi2"].to_list()
out = []
for i in range(0, len(parts), 20):
    i_st = i 
    i_sp = i + 20
    out.append(str(i).zfill(3) + " " + "".join(parts[i_st:i_sp]))

In [77]:
out

['000 一丨丶丿乀乁乙乚亅二亠人亻儿入八丷冂冖冫',
 '020 几凵刂刁刀力勹匕匚匸十卜卩㔾厂厶又口囗土',
 '040 士夂夊夕大女子宀寸小⺌⺍尢尸屮山川巛工己',
 '060 巾干幺广廴廾弋弓彐彑彡彳忄心戈户戶扌手支',
 '080 攴攵文斗斤方无旡日曰朩木欠止歹殳母比毛氏',
 '100 气氵水氺火灬爪爫父爻爿片牙牛牜⺧犭犬玄王',
 '120 玉瓜瓦甘生用田疋𤴔疒癶白皮皿目矛矢石礻示',
 '140 禸禾穴立⺮竹米纟糹 糸缶网罒罓羊⺶⺷羽耂老而',
 '160 耒耳聿月肉臣自至臼舌舛舟艮色艹虍虫血行衤',
 '180 衣覀西见見 角讠訁 言谷豆豕豸贝貝 赤走足身车車 辛辰辶',
 '200 阝邑酉釆里钅釒 金长長 门門 阜⻏隶隹雨青非面革韦韭',
 '220 音页頁 风風 飞飛 食饣飠 首香马馬 骨高髙 髟鬥鬯鬲鬼鱼魚 鸟鳥 卤鹿',
 '240 麦麻黄黍黑黹黾鼎鼓鼠鼻齐齿龙龟龠']